In [6]:
import pandas as pd
import sys

sys.path.append("../")

from lib.uid import gen_uid
from lib.clean import clean_names, clean_races, clean_sexes

In [7]:
def read_personnel_and_employment_hist():
    dfa = pd.read_csv("../../../data/GA/2022-09-27/employment-history.csv")
    return dfa


In [8]:
dfa = read_personnel_and_employment_hist()
dfa

# dfa.columns

,Unnamed: 0,OKEY,NAME,AGENCY,RANK,STATUS,START DATE,END DATE
0,NaN,O246465,AAMIR WISHAH,G1928 ELBERT COUNTY 911,COMM. OFFICER,Actively Employed in Law Enforcement,2020-06-01,0000-00-00
1,NaN,O095227,AANERUD DAMON H,G1161 CHATHAM COUNTY SHERIFFS OFFICE,JAILOR,Voluntary Resignation,1999-04-19,2000-10-07
2,NaN,O095227,AANERUD DAMON H,G1244 SAVANNAH POLICE DEPARTMENT,PEACE OFFICER,Voluntary Resignation,2000-10-09,2001-03-30
3,NaN,O095227,AANERUD DAMON H,G1177 POOLER POLICE DEPARTMENT,CORPORAL,Rank Change - Promotion,2001-03-10,2009-09-27
4,NaN,O095227,AANERUD DAMON H,G1177 POOLER POLICE DEPARTMENT,SERGEANT,Actively Employed in Law Enforcement,2009-09-28,0000-00-00
...,...,...,...,...,...,...,...,...
139655,NaN,O136250,ZYDONYK ASHBY L,G1587 NEWINGTON POLICE DEPARTMENT,CAPTAIN,Rank Change - Promotion,2020-01-20,2021-01-01
139656,NaN,O136250,ZYDONYK ASHBY L,G1587 NEWINGTON POLICE DEPARTMENT,CHIEF,Actively Employed in Law Enforcement,2021-01-01,0000-00-00
139657,NaN,O110791,ZYGAJ STEPHEN M,G1505 ATLANTA POLICE DEPARTMENT,PEACE OFFICER,Rank Change - Promotion,2001-12-18,2020-01-08
139658,NaN,O110791,ZYGAJ STEPHEN M,G1505 ATLANTA POLICE DEPARTMENT,CAPTAIN,Rank Change - Promotion,2020-01-09,2022-04-13


In [9]:


dfa = dfa.rename(columns={" NAME": "NAME", " START DATE": "hire_date", " END DATE ":"left_date", "OKEY": "OKEY_KEY"})

def split_names(df):
    names = df.NAME.str.lower().str.strip().str.extract(r"(\w+) (\w+) ?(.+)")

    df.loc[:, "last_name"] = names[0]
    df.loc[:, "middle_name"] = names[2]
    df.loc[:, "first_name"] = names[1]

    return df 

dfa = dfa.pipe(split_names)

dfa = dfa.pipe(gen_uid, ["first_name", "last_name", "middle_name"])
dfa




dfb = pd.read_csv("../../../data/GA/2022-09-27/officer_data-20231220144255.csv", encoding="latin1")
dfb = dfb.rename(columns={" SEX": "sex", " RACE ": "race", 
                              " YOB": "year_of_birth", " MIDDLE": "middle_name",
                              " LAST NAME": "last_name", " FIRST NAME": "first_name"})

dfb
    
dfb = dfb.pipe(clean_names, ["first_name", "last_name", "middle_name"])

    
dfb = dfb.pipe(gen_uid, ["first_name", "last_name", "middle_name"])
    
dfb = dfb[["year_of_birth", "race", "sex", "uid"]]

df = pd.merge(dfa, dfb, on="uid", how="left")

df["state"] = "GA"


df.sort_values("OKEY_KEY")

df.loc[:, "left_date"] = df.left_date.str.replace(r"0000-00-00", "", regex=True)

df.loc[:, "hire_date"] = df.hire_date.str.replace(r"(\w{4})-(\w{1,2})-(\w{1,2})", r"\2/\3/\1", regex=True)
df.loc[:, "left_date"] = df.left_date.str.replace(r"(\w{4})-(\w{1,2})-(\w{1,2})", r"\2/\3/\1", regex=True)

df = df[~((df.left_date.fillna("") == ""))]

df.left_date.unique()


df = df[df.duplicated(subset=["OKEY_KEY"], keep=False)]

df = df.rename(columns={" AGENCY": "agency"})

df = df.pipe(gen_uid, ["first_name", "middle_name", "last_name", "agency"])

events_df = df[["uid", "agency", "state", "hire_date", "left_date"]]


# Unpivot the DataFrame
events_df = events_df.melt(id_vars=['uid', 'agency', 'state'], 
                  value_vars=['hire_date', "left_date"], 
                  var_name='event_type', 
                  value_name='event_date')

# Generate event_uid using gen_uid function
events_df = events_df.pipe(gen_uid, ['uid', 'event_type', 'event_date'], "event_uid")

events_df.event_type.value_counts()

events_df

# events_df.to_csv("./../../../data/GA/match/events_georgia_dropped_na_left_date.csv", index=False)

,uid,agency,state,event_type,event_date,event_uid
0,1ecd52b930590412dbceaceaa97a1846,G1161 CHATHAM COUNTY SHERIFFS OFFICE,GA,hire_date,04/19/1999,6e4fa0a0c333c1f05bf54cc1db5df6f9
1,a8d978d99f97b0a827a097556bcc13ed,G1244 SAVANNAH POLICE DEPARTMENT,GA,hire_date,10/09/2000,ab94d859ff4dcd0d3663960e25ee2ff4
2,50c573910e56846b39df32389989a8e0,G1177 POOLER POLICE DEPARTMENT,GA,hire_date,03/10/2001,52755366a16785d593b0bfd19d76049c
3,9ed903f0c60ed2b59cf9732bddca10b9,G1423 JENKINS COUNTY SHERIFFS OFFICE,GA,hire_date,04/09/2009,84a8d62ed4a904127dff2abf52b82a32
4,9ed903f0c60ed2b59cf9732bddca10b9,G1423 JENKINS COUNTY SHERIFFS OFFICE,GA,hire_date,01/01/2012,9685d48d77470db86070f89fd19460dd
...,...,...,...,...,...,...
153093,695120a6b3ed96231064d9b7ba26c840,G1313 SCREVEN COUNTY SHERIFFS OFFICE,GA,left_date,07/01/2022,fa11a723a01afe11c68084dde42f3ea5
153094,8ecfb825cfa0d3c5cd4108d51b03d602,G1587 NEWINGTON POLICE DEPARTMENT,GA,left_date,01/19/2020,a396cf8471476a5b674bdd16accbcc45
153095,8ecfb825cfa0d3c5cd4108d51b03d602,G1587 NEWINGTON POLICE DEPARTMENT,GA,left_date,01/01/2021,6b788563c881be44c0946b518cc39832
153096,55526e61d2cb0314c9e8bfd2c5455b5d,G1505 ATLANTA POLICE DEPARTMENT,GA,left_date,01/08/2020,928fd11d5a5469467ea622936a774abb


In [10]:
# dup_uids_list = [x for x in events_df["uid"]]

per_df = df[["first_name", "middle_name", "last_name", "agency", "uid", "year_of_birth", "race", "sex", "state", "OKEY_KEY"]]

# per_df = per_df[per_df.uid.isin(dup_uids_list)]

per_df

# per_df.to_csv("./../../../data/GA/match/personnel_georgia_dropped_na_left_date.csv", index=False)

,first_name,middle_name,last_name,agency,uid,year_of_birth,race,sex,state,OKEY_KEY
1,damon,h,aanerud,G1161 CHATHAM COUNTY SHERIFFS OFFICE,1ecd52b930590412dbceaceaa97a1846,1972.0,NaN,,GA,O095227
2,damon,h,aanerud,G1244 SAVANNAH POLICE DEPARTMENT,a8d978d99f97b0a827a097556bcc13ed,1972.0,NaN,,GA,O095227
3,damon,h,aanerud,G1177 POOLER POLICE DEPARTMENT,50c573910e56846b39df32389989a8e0,1972.0,NaN,,GA,O095227
11,wesley,alan,aaron,G1423 JENKINS COUNTY SHERIFFS OFFICE,9ed903f0c60ed2b59cf9732bddca10b9,1977.0,NaN,,GA,O152866
12,wesley,alan,aaron,G1423 JENKINS COUNTY SHERIFFS OFFICE,9ed903f0c60ed2b59cf9732bddca10b9,1977.0,NaN,,GA,O152866
...,...,...,...,...,...,...,...,...,...,...
140630,ashby,l,zydonyk,G1313 SCREVEN COUNTY SHERIFFS OFFICE,695120a6b3ed96231064d9b7ba26c840,1985.0,NaN,,GA,O136250
140631,ashby,l,zydonyk,G1587 NEWINGTON POLICE DEPARTMENT,8ecfb825cfa0d3c5cd4108d51b03d602,1985.0,NaN,,GA,O136250
140632,ashby,l,zydonyk,G1587 NEWINGTON POLICE DEPARTMENT,8ecfb825cfa0d3c5cd4108d51b03d602,1985.0,NaN,,GA,O136250
140634,stephen,m,zygaj,G1505 ATLANTA POLICE DEPARTMENT,55526e61d2cb0314c9e8bfd2c5455b5d,1962.0,NaN,,GA,O110791
